In [60]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist

import pandas as pd

from expected_min_dist import expected_dist_conf, gen_centers


In [71]:
def greedy_step(servers, request):
    # Find the smallest distance between the servers and the point
    min_idx = -1
    min_dist = float('inf')
    for i, server in enumerate(servers):
        request_dist = dist.euclidean(server, request)
        # Store if new min
        if request_dist < min_dist:
            min_dist = request_dist
            min_idx = i
    return min_idx, min_dist

def heuristic_step(servers, request, nu=1.):
    # Find the smallest distance between the servers and the point
    min_idx = -1
    min_dist = float('inf')
    min_loss = float('inf')
    for i, server in enumerate(servers):
        request_dist = dist.euclidean(server, request)
        # Suppose we move the server, this is the new config
        future_servers = servers[: i] + [server] + servers[i:]
        # Compute the future expected minimum distance
        exp_dist = expected_dist_conf(future_servers, draw=False)
        # This is the heuristic loss function
        loss = request_dist + nu*exp_dist
        # Store minimum
        if loss < min_loss:
            min_loss = loss
            min_dist = request_dist
            min_idx = i
    return min_idx, min_dist

In [72]:
# Run a kserver algorithm
def _run_kserver(alg_step, servers, requests, **kwargs):
    # Copy list
    cur_servers = list(servers)
    total_dist = 0.
    for request in requests:
        # Compute next step
        idx, min_dist = alg_step(cur_servers, request, **kwargs)
        total_dist += min_dist
        # Take the step
        cur_servers[idx] = request
    return total_dist


def greedy_kserver(servers, requests):
    return _run_kserver(greedy_step, servers=servers, requests=requests)

def heuristic_kserver(servers, pts, nu=1.):
    return _run_kserver(heuristic_step, servers=servers, requests=requests, nu=nu)



In [108]:
num_requests = 100
num_servers = 15

servers = gen_centers(num_servers)
requests = gen_centers(num_requests)

In [111]:
greedy_cost = greedy_kserver(servers, requests)

for nu in np.linspace(0, num_servers, 10):
    print "nu: ",nu
    print heuristic_kserver(servers, requests, nu=nu) / greedy_cost
    print ""

nu:  0.0
1.0

nu:  1.66666666667
0.97958191281

nu:  3.33333333333
0.986646867335

nu:  5.0
1.00354864314

nu:  6.66666666667
1.01788842475

nu:  8.33333333333
1.01788842475

nu:  10.0
1.03983351176

nu:  11.6666666667
1.07601148925

nu:  13.3333333333
1.10524990596

nu:  15.0
1.05934437081



In [110]:
np.log(num_servers)

2.7080502011022101